In [5]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter, Gate
from qiskit.circuit.library import RXXGate, RYYGate
import matplotlib.pyplot as plt

# Set matplotlib backend explicitly
plt.switch_backend('Agg')

class XYMixerBlock(Gate):
    """Custom U_XY gate that combines RXX(β) and RYY(β)"""
    
    def __init__(self, beta):
        # Initialize the gate with 2 qubits, the parameter, and a name
        super().__init__('U_XY', 2, [beta])
        # Explicitly set the label to use LaTeX mathematical notation
        self.label = r'$U_{XY}$'
    
    def _define(self):
        """Define the gate in terms of standard gates"""
        # Create a quantum circuit that implements RXX(β) + RYY(β)
        qc = QuantumCircuit(2)
        beta = self.params[0]
        qc.rxx(beta, 0, 1)
        qc.ryy(beta, 0, 1)
        self.definition = qc

def create_six_qubit_all_to_all_uxy_circuit_aligned(spacing='auto'):
    """Create a 6-qubit circuit with U_XY gates in sequential order but with proper spacing
    
    Args:
        spacing (str): Spacing strategy
            - 'auto': Automatic spacing with identity gates
            - 'barriers': Use barriers for alignment
            - 'none': No spacing (original dense layout)
    """
    qc = QuantumCircuit(6)
    
    # Define β as a parameter
    beta = Parameter('β')
    xy_mixer = XYMixerBlock(beta)
    
    # Generate all possible pairs in sequential order
    pairs = []
    for i in range(6):
        for j in range(i+1, 6):
            pairs.append((i, j))
    
    if spacing == 'auto':
        # Add strategic identity gates to align the circuit properly
        for pair_idx, (i, j) in enumerate(pairs):
            # Apply the U_XY gate
            qc.append(xy_mixer, [i, j])
            
            # Add identity gates on unused qubits for better alignment
            # This prevents gates from overlapping visually
            if pair_idx < len(pairs) - 1:  # Don't add after the last gate
                # Find qubits not involved in current or next gate
                next_i, next_j = pairs[pair_idx + 1] if pair_idx + 1 < len(pairs) else (i, j)
                used_qubits = {i, j, next_i, next_j}
                
                for qubit in range(6):
                    if qubit not in used_qubits:
                        qc.i(qubit)  # Identity gate as spacer
                        
    elif spacing == 'barriers':
        # Use barriers to separate groups of gates
        gates_per_group = 3
        for idx, (i, j) in enumerate(pairs):
            qc.append(xy_mixer, [i, j])
            
            # Add barrier every few gates for visual separation
            if (idx + 1) % gates_per_group == 0 and idx < len(pairs) - 1:
                qc.barrier()
                
    else:  # spacing == 'none'
        # Original dense layout
        for i, j in pairs:
            qc.append(xy_mixer, [i, j])
    
    print(f"Applied U_XY gates in sequential order with '{spacing}' spacing:")
    print(f"Pairs: {pairs}")
    return qc

def create_six_qubit_all_to_all_uxy_circuit(ordering='sequential'):
    """Create a 6-qubit circuit with U_XY gates on all possible pairs (all-to-all connectivity)
    
    Args:
        ordering (str): Gate ordering strategy
            - 'sequential': (0,1), (0,2), ..., (0,5), (1,2), ..., (4,5) [default]
            - 'by_distance': Group by qubit separation distance
            - 'layered': Apply gates in parallel-friendly layers
            - 'custom': Use a custom predefined order
    """
    qc = QuantumCircuit(6)
    
    # Define β as a parameter
    beta = Parameter('β')
    xy_mixer = XYMixerBlock(beta)
    
    # Generate all possible pairs
    all_pairs = []
    for i in range(6):
        for j in range(i+1, 6):
            all_pairs.append((i, j))
    
    if ordering == 'sequential':
        # Default: sequential order
        pairs = all_pairs
        
    elif ordering == 'by_distance':
        # Group by distance between qubits
        pairs = []
        for distance in range(1, 6):  # distances 1, 2, 3, 4, 5
            for i in range(6):
                j = i + distance
                if j < 6:
                    pairs.append((i, j))
                    
    elif ordering == 'layered':
        # Arrange in layers that can be executed in parallel
        # Layer 1: (0,1), (2,3), (4,5)
        # Layer 2: (0,2), (1,3), (4,6) - but we only have 6 qubits
        # etc.
        pairs = []
        # Layer 1: qubits with distance 1, non-overlapping
        pairs.extend([(0,1), (2,3), (4,5)])
        # Layer 2: fill in remaining adjacent pairs
        pairs.extend([(1,2), (3,4)])
        # Layer 3: distance 2 pairs
        pairs.extend([(0,2), (1,3), (2,4), (3,5)])
        # Layer 4: remaining distance 2
        pairs.extend([(0,3), (1,4), (2,5)])
        # Layer 5: distance 3+ pairs
        pairs.extend([(0,4), (1,5)])
        # Layer 6: final long-distance pair
        pairs.extend([(0,5)])
        
    elif ordering == 'custom':
        # Example custom ordering - you can modify this
        pairs = [
            (0,5), (1,4), (2,3),  # Long distance first
            (0,1), (1,2), (2,3), (3,4), (4,5),  # Chain
            (0,2), (1,3), (2,4), (3,5),  # Skip one
            (0,3), (1,4), (2,5),  # Skip two
            (0,4)  # Remaining
        ]
        # Remove duplicates while preserving order
        seen = set()
        pairs = [pair for pair in pairs if not (pair in seen or seen.add(pair))]
    
    # Apply gates in the specified order
    for i, j in pairs:
        qc.append(xy_mixer, [i, j])
    
    print(f"Applied U_XY gates in '{ordering}' order to {len(pairs)} pairs:")
    print(f"Order: {pairs}")
    return qc

def create_three_qubit_uxy_circuit():
    """Create a 3-qubit circuit with U_XY gates on each pair"""
    qc = QuantumCircuit(3)
    
    # Define β as a parameter
    beta = Parameter('β')
    
    # Create U_XY gate instances
    xy_mixer = XYMixerBlock(beta)
    
    # Apply U_XY gate on each pair of qubits
    # Pair (0,1)
    qc.append(xy_mixer, [0, 1])
    
    # Pair (1,2)
    qc.append(xy_mixer, [1, 2])
    
    # Pair (0,2)
    qc.append(xy_mixer, [0, 2])
    
    return qc

# Create the circuits
circuit_3qubit = create_three_qubit_uxy_circuit()

# Create 6-qubit circuits with different approaches
circuit_6qubit_aligned = create_six_qubit_all_to_all_uxy_circuit_aligned('auto')
circuit_6qubit_barriers = create_six_qubit_all_to_all_uxy_circuit_aligned('barriers')
circuit_6qubit_dense = create_six_qubit_all_to_all_uxy_circuit_aligned('none')

# Print text representations
print("3-Qubit Circuit with U_XY gates:")
print(circuit_3qubit.draw())

print("\n" + "="*70)
print("6-Qubit Sequential U_XY Circuit (Auto-aligned with invisible spacers):")
print(circuit_6qubit_aligned.draw())

print("\n" + "="*70)
print("6-Qubit Sequential U_XY Circuit (With barriers):")
print(circuit_6qubit_barriers.draw())

print("\n" + "="*70)
print("6-Qubit Sequential U_XY Circuit (Dense - original):")
print(circuit_6qubit_dense.draw())

# Create a simple 2-qubit XY Mixer Block circuit for standalone diagram
print("\nCreating standalone 2-qubit U_XY gate diagram...")
beta_standalone = Parameter('β')
xy_mixer_standalone = XYMixerBlock(beta_standalone)
standalone_circuit = QuantumCircuit(2)
standalone_circuit.append(xy_mixer_standalone, [0, 1])

print("2-Qubit U_XY Gate:")
print(standalone_circuit.draw())

# Save circuit diagram function
def save_circuit_diagram(circuit, filename, title):
    """Save circuit diagram as an image file"""
    try:
        # Create the figure using qiskit's draw method
        fig = circuit.draw(output='mpl', style='iqp')
        
        # Adjust layout
        fig.tight_layout()
        
        # Save the figure
        fig.savefig(filename, dpi=300, bbox_inches='tight')
        
        # Close the figure to free memory
        plt.close(fig)
        
        print(f"Saved {title} diagram as {filename}")
        
    except Exception as e:
        print(f"Error saving diagram: {e}")

# Save circuit diagrams
save_circuit_diagram(circuit_3qubit, 'three_qubit_uxy_gates.png', 
                    '3-Qubit Circuit: U_XY Gates')

save_circuit_diagram(circuit_6qubit_aligned, 'six_qubit_uxy_aligned.png', 
                    '6-Qubit Sequential U_XY Circuit (Auto-aligned)')

save_circuit_diagram(circuit_6qubit_barriers, 'six_qubit_uxy_barriers.png', 
                    '6-Qubit Sequential U_XY Circuit (With barriers)')

save_circuit_diagram(circuit_6qubit_dense, 'six_qubit_uxy_dense.png', 
                    '6-Qubit Sequential U_XY Circuit (Dense)')

save_circuit_diagram(standalone_circuit, 'uxy_gate_2qubit.png', 
                    '2-Qubit U_XY Gate')

# Check if files were created
import os
files_to_check = [
    'three_qubit_uxy_gates.png',
    'six_qubit_uxy_aligned.png',
    'six_qubit_uxy_barriers.png', 
    'six_qubit_uxy_dense.png',
    'uxy_gate_2qubit.png'
]

print("\nFile check results:")
for filename in files_to_check:
    if os.path.exists(filename):
        file_size = os.path.getsize(filename)
        print(f"✓ {filename} - Created successfully ({file_size} bytes)")
    else:
        print(f"✗ {filename} - Not found")

# Show the decomposition of the XY Mixer Block
print("\nU_XY Gate Decomposition:")
print("=" * 50)
beta_test = Parameter('β')
xy_mixer_test = XYMixerBlock(beta_test)
test_circuit = QuantumCircuit(2)
test_circuit.append(xy_mixer_test, [0, 1])

print("U_XY gate on 2 qubits:")
print(test_circuit.draw())

print("\nDecomposed into standard gates:")
decomposed = test_circuit.decompose()
print(decomposed.draw())

print("\nCircuit Analysis:")
print("=" * 50)
print("• Each U_XY gate combines RXX(β) + RYY(β)")
print("• 3-qubit circuit: 3 U_XY gates covering all pairs")
print("• 6-qubit circuit: 15 U_XY gates in sequential order")
print("• Alignment strategies:")
print("  - Auto-aligned: Identity gates as invisible spacers")
print("  - Barriers: Visual separation with barriers")
print("  - Dense: Original compact layout")
print("• Sequential ordering: (0,1), (0,2), ..., (4,5)")
print("• Auto-alignment prevents gate overlap and improves readability")
print("• Perfect for QAOA XY mixer Hamiltonian evolution")

print("\nDone! You now have properly aligned sequential U_XY circuits.")
print("Generated: 2-qubit, 3-qubit, and 6-qubit (3 alignment options)!")
    
    def _define(self):
        """Define the gate in terms of standard gates"""
        # Create a quantum circuit that implements RXX(β) + RYY(β)
        qc = QuantumCircuit(2)
        beta = self.params[0]
        qc.rxx(beta, 0, 1)
        qc.ryy(beta, 0, 1)
        self.definition = qc

def create_three_qubit_xy_mixer_circuit():
    """Create a 3-qubit circuit with XY Mixer Block gates on each pair"""
    qc = QuantumCircuit(3)
    
    # Define β as a parameter
    beta = Parameter('β')
    
    # Create XY Mixer Block instances
    xy_mixer = XYMixerBlock(beta)
    
    # Apply XY Mixer Block on each pair of qubits
    # Pair (0,1)
    qc.append(xy_mixer, [0, 1])
    
    # Pair (1,2)
    qc.append(xy_mixer, [1, 2])
    
    # Pair (0,2)
    qc.append(xy_mixer, [0, 2])
    
    return qc

# Create the main circuit
circuit_same_params = create_three_qubit_xy_mixer_circuit()

# Print text representation
print("3-Qubit Circuit with XY Mixer Block gates (same parameter β):")
print(circuit_same_params.draw())

# Create a simple 2-qubit XY Mixer Block circuit for standalone diagram
print("\nCreating standalone 2-qubit XY Mixer Block diagram...")
beta_standalone = Parameter('β')
xy_mixer_standalone = XYMixerBlock(beta_standalone)
standalone_circuit = QuantumCircuit(2)
standalone_circuit.append(xy_mixer_standalone, [0, 1])

print("2-Qubit XY Mixer Block:")
print(standalone_circuit.draw())

# Save circuit diagram function
def save_circuit_diagram(circuit, filename, title):
    """Save circuit diagram as an image file"""
    try:
        # Create the figure using qiskit's draw method
        fig = circuit.draw(output='mpl', style='iqp')
        
        # Adjust layout
        fig.tight_layout()
        
        # Save the figure
        fig.savefig(filename, dpi=300, bbox_inches='tight')
        
        # Close the figure to free memory
        plt.close(fig)
        
        print(f"Saved {title} diagram as {filename}")
        
    except Exception as e:
        print(f"Error saving diagram: {e}")

# Save circuit diagrams
save_circuit_diagram(circuit_same_params, 'three_qubit_xy_mixer_same_beta.png', 
                    '3-Qubit Circuit: XY Mixer Block(β) on All Pairs')

save_circuit_diagram(standalone_circuit, 'xy_mixer_block_2qubit.png', 
                    '2-Qubit XY Mixer Block')

# Check if files were created
import os
files_to_check = [
    'three_qubit_xy_mixer_same_beta.png',
    'xy_mixer_block_2qubit.png'
]

print("\nFile check results:")
for filename in files_to_check:
    if os.path.exists(filename):
        file_size = os.path.getsize(filename)
        print(f"✓ {filename} - Created successfully ({file_size} bytes)")
    else:
        print(f"✗ {filename} - Not found")

# Show the decomposition of the XY Mixer Block
print("\nXY Mixer Block Decomposition:")
print("=" * 50)
beta_test = Parameter('β')
xy_mixer_test = XYMixerBlock(beta_test)
test_circuit = QuantumCircuit(2)
test_circuit.append(xy_mixer_test, [0, 1])

print("XY Mixer Block on 2 qubits:")
print(test_circuit.draw())

print("\nDecomposed into standard gates:")
decomposed = test_circuit.decompose()
print(decomposed.draw())

print("\nCircuit Analysis:")
print("=" * 50)
print("• Each XY Mixer Block combines RXX(β) + RYY(β)")
print("• Total of 3 XY Mixer Blocks covering pairs: (0,1), (1,2), (0,2)")
print("• Much cleaner visualization than 6 separate gates")
print("• Maintains the same functionality as before")
print("• Perfect for QAOA XY mixer Hamiltonian evolution")
print("• Preserves excitation number in the system")

print("\nDone! You now have clean XY Mixer Block representations of your circuits.")

SyntaxError: unterminated string literal (detected at line 17) (772737757.py, line 17)

In [10]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter, Gate
from qiskit.circuit.library import RXXGate, RYYGate
import matplotlib.pyplot as plt

# Set matplotlib backend explicitly
plt.switch_backend('Agg')

class XYMixerBlock(Gate):
    """Custom U_XY gate that combines RXX(β) and RYY(β)"""
    
    def __init__(self, beta):
        # Initialize the gate with 2 qubits, the parameter, and a name
        super().__init__('U_XY', 2, [beta])
        # Explicitly set the label to use mathematical notation
        self.label = 'U_XY'
    
    def _define(self):
        """Define the gate in terms of standard gates"""
        # Create a quantum circuit that implements RXX(β) + RYY(β)
        qc = QuantumCircuit(2)
        beta = self.params[0]
        qc.rxx(beta, 0, 1)
        qc.ryy(beta, 0, 1)
        self.definition = qc

def create_six_qubit_all_to_all_uxy_circuit_aligned(spacing='auto'):
    """Create a 6-qubit circuit with U_XY gates in sequential order but with proper spacing"""
    qc = QuantumCircuit(6)
    
    # Define β as a parameter
    beta = Parameter('β')
    xy_mixer = XYMixerBlock(beta)
    
    # Generate pairs in the exact order requested:
    # First all gates from 0: (0,1), (0,2), (0,3), (0,4), (0,5)
    # Then all from 1 (except 0): (1,2), (1,3), (1,4), (1,5)  
    # Then all from 2 (except 0,1): (2,3), (2,4), (2,5)
    # etc.
    pairs = []
    for i in range(6):
        for j in range(i+1, 6):
            pairs.append((i, j))
    
    if spacing == 'invisible_barriers':
        # Group gates by first qubit and add invisible spacing between groups
        current_first_qubit = 0
        
        for pair_idx, (i, j) in enumerate(pairs):
            # Add invisible spacing when we switch to a new first qubit
            if i != current_first_qubit:
                # Add identity gates on all qubits for invisible spacing
                for qubit in range(6):
                    qc.id(qubit)
                current_first_qubit = i
            
            # Apply the U_XY gate
            qc.append(xy_mixer, [i, j])
    
    elif spacing == 'auto':
        # Add strategic identity gates to align the circuit properly
        for pair_idx, (i, j) in enumerate(pairs):
            # Apply the U_XY gate
            qc.append(xy_mixer, [i, j])
            
            # Add identity gates on unused qubits for better alignment
            if pair_idx < len(pairs) - 1:  # Don't add after the last gate
                # Find qubits not involved in current or next gate
                next_i, next_j = pairs[pair_idx + 1] if pair_idx + 1 < len(pairs) else (i, j)
                used_qubits = {i, j, next_i, next_j}
                
                for qubit in range(6):
                    if qubit not in used_qubits:
                        qc.id(qubit)  # Identity gate as spacer
                        
    elif spacing == 'barriers':
        # Use barriers to separate groups of gates - group by first qubit
        current_first_qubit = 0
        
        for pair_idx, (i, j) in enumerate(pairs):
            # Add barrier when we switch to a new first qubit  
            if i != current_first_qubit and pair_idx > 0:
                qc.barrier()
                current_first_qubit = i
            
            # Apply the U_XY gate
            qc.append(xy_mixer, [i, j])
                
    else:  # spacing == 'none'
        # Original dense layout
        for i, j in pairs:
            qc.append(xy_mixer, [i, j])
    
    print(f"Applied U_XY gates in sequential order with spacing: {spacing}")
    print("Gate order by first qubit:")
    current_first = 0
    group = []
    for i, j in pairs:
        if i != current_first:
            if group:
                print(f"  Qubit {current_first}: {group}")
            group = []
            current_first = i
        group.append((i, j))
    if group:
        print(f"  Qubit {current_first}: {group}")
    
    return qc

def create_three_qubit_uxy_circuit():
    """Create a 3-qubit circuit with U_XY gates on each pair"""
    qc = QuantumCircuit(3)
    
    # Define β as a parameter
    beta = Parameter('β')
    
    # Create U_XY gate instances
    xy_mixer = XYMixerBlock(beta)
    
    # Apply U_XY gate on each pair of qubits
    qc.append(xy_mixer, [0, 1])  # Pair (0,1)
    qc.append(xy_mixer, [1, 2])  # Pair (1,2)
    qc.append(xy_mixer, [0, 2])  # Pair (0,2)
    
    return qc

# Create the circuits
circuit_3qubit = create_three_qubit_uxy_circuit()

# Create 6-qubit circuits with different approaches
circuit_6qubit_invisible = create_six_qubit_all_to_all_uxy_circuit_aligned('invisible_barriers')
circuit_6qubit_barriers = create_six_qubit_all_to_all_uxy_circuit_aligned('barriers')
circuit_6qubit_dense = create_six_qubit_all_to_all_uxy_circuit_aligned('none')

# Print text representations
print("3-Qubit Circuit with U_XY gates:")
print(circuit_3qubit.draw())

print("\nSequential 6-Qubit U_XY Circuit (Invisible barriers - grouped by first qubit):")
print(circuit_6qubit_invisible.draw())

print("\nSequential 6-Qubit U_XY Circuit (Visible barriers):")
print(circuit_6qubit_barriers.draw())

print("\nSequential 6-Qubit U_XY Circuit (Dense - original):")
print(circuit_6qubit_dense.draw())

# Create a simple 2-qubit U_XY gate for standalone diagram
print("\nCreating standalone 2-qubit U_XY gate diagram...")
beta_standalone = Parameter('β')
xy_mixer_standalone = XYMixerBlock(beta_standalone)
standalone_circuit = QuantumCircuit(2)
standalone_circuit.append(xy_mixer_standalone, [0, 1])

print("2-Qubit U_XY Gate:")
print(standalone_circuit.draw())

# Save circuit diagram function
def save_circuit_diagram(circuit, filename, title):
    """Save circuit diagram as an image file"""
    try:
        # Create the figure using qiskit's draw method
        fig = circuit.draw(output='mpl', style='iqp')
        
        # Adjust layout
        fig.tight_layout()
        
        # Save the figure
        fig.savefig(filename, dpi=300, bbox_inches='tight')
        
        # Close the figure to free memory
        plt.close(fig)
        
        print(f"Saved diagram as {filename}")
        
    except Exception as e:
        print(f"Error saving diagram: {e}")

# Save circuit diagrams
save_circuit_diagram(circuit_3qubit, 'three_qubit_uxy_gates.png', 
                    '3-Qubit Circuit: U_XY Gates')

save_circuit_diagram(circuit_6qubit_invisible, 'six_qubit_uxy_invisible_barriers.png', 
                    '6-Qubit Sequential U_XY Circuit (Invisible barriers)')

save_circuit_diagram(circuit_6qubit_barriers, 'six_qubit_uxy_barriers.png', 
                    '6-Qubit Sequential U_XY Circuit (Visible barriers)')

save_circuit_diagram(circuit_6qubit_dense, 'six_qubit_uxy_dense.png', 
                    '6-Qubit Sequential U_XY Circuit (Dense)')

save_circuit_diagram(standalone_circuit, 'uxy_gate_2qubit.png', 
                    '2-Qubit U_XY Gate')

# Check if files were created
import os
files_to_check = [
    'three_qubit_uxy_gates.png',
    'six_qubit_uxy_invisible_barriers.png',
    'six_qubit_uxy_barriers.png', 
    'six_qubit_uxy_dense.png',
    'uxy_gate_2qubit.png'
]

print("\nFile check results:")
for filename in files_to_check:
    if os.path.exists(filename):
        file_size = os.path.getsize(filename)
        print(f"✓ {filename} - Created successfully ({file_size} bytes)")
    else:
        print(f"✗ {filename} - Not found")

# Show the decomposition of the U_XY gate
print("\nU_XY Gate Decomposition:")
print("=" * 50)
beta_test = Parameter('β')
xy_mixer_test = XYMixerBlock(beta_test)
test_circuit = QuantumCircuit(2)
test_circuit.append(xy_mixer_test, [0, 1])

print("U_XY gate on 2 qubits:")
print(test_circuit.draw())

print("\nDecomposed into standard gates:")
decomposed = test_circuit.decompose()
print(decomposed.draw())

print("\nCircuit Analysis:")
print("=" * 50)
print("• Each U_XY gate combines RXX(β) + RYY(β)")
print("• 3-qubit circuit: 3 U_XY gates covering all pairs")
print("• 6-qubit circuit: 15 U_XY gates in perfect sequential order:")
print("  - Qubit 0: (0,1), (0,2), (0,3), (0,4), (0,5)")
print("  - Qubit 1: (1,2), (1,3), (1,4), (1,5)")
print("  - Qubit 2: (2,3), (2,4), (2,5)")
print("  - Qubit 3: (3,4), (3,5)")
print("  - Qubit 4: (4,5)")
print("• Alignment strategies:")
print("  - Invisible barriers: Identity gates for spacing between qubit groups")
print("  - Visible barriers: Barriers to show logical grouping")
print("  - Dense: Original compact layout")
print("• Invisible barriers provide clean separation without visual clutter")
print("• Perfect for QAOA XY mixer Hamiltonian evolution")

print("\nDone! You now have perfectly ordered sequential U_XY circuits.")
print("Generated: 2-qubit, 3-qubit, and 6-qubit (with invisible barriers)!")

Applied U_XY gates in sequential order with spacing: invisible_barriers
Gate order by first qubit:
  Qubit 0: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]
  Qubit 1: [(1, 2), (1, 3), (1, 4), (1, 5)]
  Qubit 2: [(2, 3), (2, 4), (2, 5)]
  Qubit 3: [(3, 4), (3, 5)]
  Qubit 4: [(4, 5)]
Applied U_XY gates in sequential order with spacing: barriers
Gate order by first qubit:
  Qubit 0: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]
  Qubit 1: [(1, 2), (1, 3), (1, 4), (1, 5)]
  Qubit 2: [(2, 3), (2, 4), (2, 5)]
  Qubit 3: [(3, 4), (3, 5)]
  Qubit 4: [(4, 5)]
Applied U_XY gates in sequential order with spacing: none
Gate order by first qubit:
  Qubit 0: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]
  Qubit 1: [(1, 2), (1, 3), (1, 4), (1, 5)]
  Qubit 2: [(2, 3), (2, 4), (2, 5)]
  Qubit 3: [(3, 4), (3, 5)]
  Qubit 4: [(4, 5)]
3-Qubit Circuit with U_XY gates:
     ┌──────────┐            ┌──────────┐
q_0: ┤0         ├────────────┤0         ├
     │  U_xy(β) │┌──────────┐│          │
q_1: ┤1         ├┤0         

In [12]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter, Gate
from qiskit.circuit.library import RXXGate, RYYGate
import matplotlib.pyplot as plt

# Set matplotlib backend explicitly
plt.switch_backend('Agg')

class XYMixerBlock(Gate):
    """Custom U_XY gate that combines RXX(β) and RYY(β)"""
    
    def __init__(self, beta):
        # Initialize the gate with 2 qubits, the parameter, and a name
        super().__init__('U_XY', 2, [beta])
        # Explicitly set the label to use mathematical notation
        self.label = r'$U_{XY}}$'
    
    def _define(self):
        """Define the gate in terms of standard gates"""
        # Create a quantum circuit that implements RXX(β) + RYY(β)
        qc = QuantumCircuit(2)
        beta = self.params[0]
        qc.rxx(beta, 0, 1)
        qc.ryy(beta, 0, 1)
        self.definition = qc

def create_six_qubit_all_to_all_uxy_circuit_aligned(spacing='auto'):
    """Create a 6-qubit circuit with U_XY gates in sequential order but with proper spacing"""
    qc = QuantumCircuit(6)
    
    # Define β as a parameter
    beta = Parameter('β')
    xy_mixer = XYMixerBlock(beta)
    
    # Generate pairs in the exact order requested:
    # First all gates from 0: (0,1), (0,2), (0,3), (0,4), (0,5)
    # Then all from 1 (except 0): (1,2), (1,3), (1,4), (1,5)  
    # Then all from 2 (except 0,1): (2,3), (2,4), (2,5)
    # etc.
    pairs = []
    for i in range(6):
        for j in range(i+1, 6):
            pairs.append((i, j))
    
    if spacing == 'invisible_barriers':
        # Group gates by first qubit and add invisible spacing between groups
        current_first_qubit = 0
        
        for pair_idx, (i, j) in enumerate(pairs):
            # Add invisible spacing when we switch to a new first qubit
            if i != current_first_qubit:
                # Add identity gates on all qubits for invisible spacing
                for qubit in range(6):
                    qc.id(qubit)
                current_first_qubit = i
            
            # Apply the U_XY gate
            qc.append(xy_mixer, [i, j])
    
    elif spacing == 'auto':
        # Add strategic identity gates to align the circuit properly
        for pair_idx, (i, j) in enumerate(pairs):
            # Apply the U_XY gate
            qc.append(xy_mixer, [i, j])
            
            # Add identity gates on unused qubits for better alignment
            if pair_idx < len(pairs) - 1:  # Don't add after the last gate
                # Find qubits not involved in current or next gate
                next_i, next_j = pairs[pair_idx + 1] if pair_idx + 1 < len(pairs) else (i, j)
                used_qubits = {i, j, next_i, next_j}
                
                for qubit in range(6):
                    if qubit not in used_qubits:
                        qc.id(qubit)  # Identity gate as spacer
                        
    elif spacing == 'barriers':
        # Use identity gates to create the same spacing as barriers but invisibly
        current_first_qubit = 0
        
        for pair_idx, (i, j) in enumerate(pairs):
            # Add invisible spacing (identity on all qubits) when we switch to a new first qubit  
            if i != current_first_qubit and pair_idx > 0:
                # Add identity gates on all qubits for spacing (same effect as barriers)
                for qubit in range(6):
                    qc.id(qubit)
                current_first_qubit = i
            
            # Apply the U_XY gate
            qc.append(xy_mixer, [i, j])
            
    elif spacing == 'visible_barriers':
        # Use actual visible barriers to separate groups of gates
        current_first_qubit = 0
        
        for pair_idx, (i, j) in enumerate(pairs):
            # Add barrier when we switch to a new first qubit  
            if i != current_first_qubit and pair_idx > 0:
                qc.barrier()
                current_first_qubit = i
            
            # Apply the U_XY gate
            qc.append(xy_mixer, [i, j])
                
    else:  # spacing == 'none'
        # Original dense layout
        for i, j in pairs:
            qc.append(xy_mixer, [i, j])
    
    print(f"Applied U_XY gates in sequential order with spacing: {spacing}")
    print("Gate order by first qubit:")
    current_first = 0
    group = []
    for i, j in pairs:
        if i != current_first:
            if group:
                print(f"  Qubit {current_first}: {group}")
            group = []
            current_first = i
        group.append((i, j))
    if group:
        print(f"  Qubit {current_first}: {group}")
    
    return qc

def create_three_qubit_uxy_circuit():
    """Create a 3-qubit circuit with U_XY gates on each pair"""
    qc = QuantumCircuit(3)
    
    # Define β as a parameter
    beta = Parameter('β')
    
    # Create U_XY gate instances
    xy_mixer = XYMixerBlock(beta)
    
    # Apply U_XY gate on each pair of qubits
    qc.append(xy_mixer, [0, 1])  # Pair (0,1)
    qc.append(xy_mixer, [1, 2])  # Pair (1,2)
    qc.append(xy_mixer, [0, 2])  # Pair (0,2)
    
    return qc

# Create the circuits
circuit_3qubit = create_three_qubit_uxy_circuit()

# Create 6-qubit circuits with different approaches  
circuit_6qubit_invisible_barriers = create_six_qubit_all_to_all_uxy_circuit_aligned('barriers')
circuit_6qubit_visible_barriers = create_six_qubit_all_to_all_uxy_circuit_aligned('visible_barriers')
circuit_6qubit_dense = create_six_qubit_all_to_all_uxy_circuit_aligned('none')

# Print text representations
print("3-Qubit Circuit with U_XY gates:")
print(circuit_3qubit.draw())

print("\nSequential 6-Qubit U_XY Circuit (Invisible barriers - perfect spacing):")
print(circuit_6qubit_invisible_barriers.draw())

print("\nSequential 6-Qubit U_XY Circuit (Visible barriers):")
print(circuit_6qubit_visible_barriers.draw())

print("\nSequential 6-Qubit U_XY Circuit (Dense - original):")
print(circuit_6qubit_dense.draw())

# Create a simple 2-qubit U_XY gate for standalone diagram
print("\nCreating standalone 2-qubit U_XY gate diagram...")
beta_standalone = Parameter('β')
xy_mixer_standalone = XYMixerBlock(beta_standalone)
standalone_circuit = QuantumCircuit(2)
standalone_circuit.append(xy_mixer_standalone, [0, 1])

print("2-Qubit U_XY Gate:")
print(standalone_circuit.draw())

# Save circuit diagram function
def save_circuit_diagram(circuit, filename, title):
    """Save circuit diagram as an image file"""
    try:
        # Create the figure using qiskit's draw method
        fig = circuit.draw(output='mpl', style='iqp')
        
        # Adjust layout
        fig.tight_layout()
        
        # Save the figure
        fig.savefig(filename, dpi=300, bbox_inches='tight')
        
        # Close the figure to free memory
        plt.close(fig)
        
        print(f"Saved diagram as {filename}")
        
    except Exception as e:
        print(f"Error saving diagram: {e}")

# Save circuit diagrams
save_circuit_diagram(circuit_3qubit, 'three_qubit_uxy_gates.png', 
                    '3-Qubit Circuit: U_XY Gates')

save_circuit_diagram(circuit_6qubit_invisible_barriers, 'six_qubit_uxy_invisible_barriers.png', 
                    '6-Qubit Sequential U_XY Circuit (Invisible barriers)')

save_circuit_diagram(circuit_6qubit_visible_barriers, 'six_qubit_uxy_visible_barriers.png', 
                    '6-Qubit Sequential U_XY Circuit (Visible barriers)')

save_circuit_diagram(circuit_6qubit_dense, 'six_qubit_uxy_dense.png', 
                    '6-Qubit Sequential U_XY Circuit (Dense)')

save_circuit_diagram(standalone_circuit, 'uxy_gate_2qubit.png', 
                    '2-Qubit U_XY Gate')

# Check if files were created
import os
files_to_check = [
    'three_qubit_uxy_gates.png',
    'six_qubit_uxy_invisible_barriers.png',
    'six_qubit_uxy_visible_barriers.png', 
    'six_qubit_uxy_dense.png',
    'uxy_gate_2qubit.png'
]

print("\nFile check results:")
for filename in files_to_check:
    if os.path.exists(filename):
        file_size = os.path.getsize(filename)
        print(f"✓ {filename} - Created successfully ({file_size} bytes)")
    else:
        print(f"✗ {filename} - Not found")

# Show the decomposition of the U_XY gate
print("\nU_XY Gate Decomposition:")
print("=" * 50)
beta_test = Parameter('β')
xy_mixer_test = XYMixerBlock(beta_test)
test_circuit = QuantumCircuit(2)
test_circuit.append(xy_mixer_test, [0, 1])

print("U_XY gate on 2 qubits:")
print(test_circuit.draw())

print("\nDecomposed into standard gates:")
decomposed = test_circuit.decompose()
print(decomposed.draw())

print("\nCircuit Analysis:")
print("=" * 50)
print("• Each U_XY gate combines RXX(β) + RYY(β)")
print("• 3-qubit circuit: 3 U_XY gates covering all pairs")
print("• 6-qubit circuit: 15 U_XY gates in perfect sequential order:")
print("  - Qubit 0: (0,1), (0,2), (0,3), (0,4), (0,5)")
print("  - Qubit 1: (1,2), (1,3), (1,4), (1,5)")
print("  - Qubit 2: (2,3), (2,4), (2,5)")
print("  - Qubit 3: (3,4), (3,5)")
print("  - Qubit 4: (4,5)")
print("• Spacing strategies:")
print("  - barriers: Identity gates create invisible spacing (recommended)")
print("  - visible_barriers: Actual barrier lines for reference")
print("  - Dense: Original compact layout")
print("• Invisible barriers provide the perfect spacing you wanted!")
print("• Perfect for QAOA XY mixer Hamiltonian evolution")

print("\nDone! You now have perfectly spaced sequential U_XY circuits with invisible barriers!")
print("Generated: 2-qubit, 3-qubit, and 6-qubit (invisible barriers like you wanted)!")

Applied U_XY gates in sequential order with spacing: barriers
Gate order by first qubit:
  Qubit 0: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]
  Qubit 1: [(1, 2), (1, 3), (1, 4), (1, 5)]
  Qubit 2: [(2, 3), (2, 4), (2, 5)]
  Qubit 3: [(3, 4), (3, 5)]
  Qubit 4: [(4, 5)]
Applied U_XY gates in sequential order with spacing: visible_barriers
Gate order by first qubit:
  Qubit 0: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]
  Qubit 1: [(1, 2), (1, 3), (1, 4), (1, 5)]
  Qubit 2: [(2, 3), (2, 4), (2, 5)]
  Qubit 3: [(3, 4), (3, 5)]
  Qubit 4: [(4, 5)]
Applied U_XY gates in sequential order with spacing: none
Gate order by first qubit:
  Qubit 0: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]
  Qubit 1: [(1, 2), (1, 3), (1, 4), (1, 5)]
  Qubit 2: [(2, 3), (2, 4), (2, 5)]
  Qubit 3: [(3, 4), (3, 5)]
  Qubit 4: [(4, 5)]
3-Qubit Circuit with U_XY gates:
     ┌───────────────┐                 ┌───────────────┐
q_0: ┤0              ├─────────────────┤0              ├
     │  $U_{XY}}$(β) │┌───────────────┐│  